In [106]:
import pandas as pd
import numpy as np
import datetime as dt

In [107]:
# Importamos el dataset de trabajo
df = pd.read_csv('./dataset/movies_dataset.csv', delimiter=',')
# Se elimina las columnas que no serán utilizadas, video,imdb_id,adult,original_title,poster_path y homepage
df = df.drop(columns = ['video','imdb_id','adult','original_title','poster_path','homepage']) 
# df.head()
print(" El dataset tiene", df.shape[0],"filas. ")


C:\Users\HP\AppData\Local\Temp\ipykernel_44512\970392396.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./dataset/movies_dataset.csv', delimiter=',')


 El dataset tiene 45466 filas. 


In [108]:
# Metodo para detectar los datos duplicados
# me sirve para ver si existen registros duplicados
df.duplicated().sum()
# Se averigua cuantos registros ("diferentes") duplicados hay.

13

In [109]:
# Se eliminan esos registros duplicados:

df = df.drop_duplicates()
# datafrane sin duplicados
df.shape

print(" El dataset queda con ", df.shape[0],"filas. ")

 El dataset queda con  45453 filas. 


In [110]:
# Se averigua cuantos valores nulos se tiene por columna:
df.isnull().sum()

print("Los campos que poseen nulos son: \n",df.isnull().sum())

Los campos que poseen nulos son: 
 belongs_to_collection    40959
budget                       0
genres                       0
id                           0
original_language           11
overview                   954
popularity                   5
production_companies         3
production_countries         3
release_date                87
revenue                      6
runtime                    263
spoken_languages             6
status                      87
tagline                  25045
title                        6
vote_average                 6
vote_count                   6
dtype: int64


In [111]:
# COLUMNA revenue
# Imputamos con el valor 0 los campos nulos de la con una revenue - recaudacion de la pelicula

df['revenue'].fillna(0, inplace=True)

df.isnull().sum()

belongs_to_collection    40959
budget                       0
genres                       0
id                           0
original_language           11
overview                   954
popularity                   5
production_companies         3
production_countries         3
release_date                87
revenue                      0
runtime                    263
spoken_languages             6
status                      87
tagline                  25045
title                        6
vote_average                 6
vote_count                   6
dtype: int64

In [112]:
# Se averigua el tipo de dato de cada columna y se decide si es el tipo de dato correcto
# Asimismo al corregir el tipo de dato, se analizan los datos cargados. budge es el presupuesto de la pelicula pasar a float
# Pero primero remplazamos otros valores.
df['budget'].replace('/ff9qCepilowshEtG2GYWwzt2bs4.jpg', 0, inplace = True)
df['budget'].replace('/zV8bHuSL6WXoD6FWogP9j4x80bL.jpg', 0, inplace = True)
df['budget'].replace('/zaSf5OG7V8X8gqFvly88zDdRm46.jpg', 0, inplace = True)

df['budget'] = df['budget'].astype(float)

df['budget'].dtype

dtype('float64')

In [113]:
# VEEEEER belongs_to_collection
# VERRRR genres

df['id'].duplicated().sum()
# Se averigua cuantos registros ("diferentes") duplicados hay. ver estooooo

17

In [114]:
# Los valores nulos del campo release date (fecha de estreno) deben eliminarse. 
# Se decide eliminar pues no serviran para las consultas en la API posteriormente.

df['release_date'].replace('1',None, inplace = True)
df['release_date'].replace('12',None, inplace = True)
df['release_date'].replace('22',None, inplace = True)
df.dropna(subset='release_date',inplace=True )

df.isnull().sum()


belongs_to_collection    40875
budget                       0
genres                       0
id                           0
original_language           11
overview                   941
popularity                   0
production_companies         0
production_countries         0
release_date                 0
revenue                      0
runtime                    246
spoken_languages             0
status                      80
tagline                  24969
title                        0
vote_average                 0
vote_count                   0
dtype: int64

In [115]:
print(" El dataset tiene", df.shape[0],"filas. ")

 El dataset tiene 45363 filas. 


In [116]:
df['release_date'].sort_values(ascending=True)

34940    1874-12-09
34937    1878-06-14
41602    1883-11-19
34933    1887-08-18
34934    1888-01-01
            ...    
44535    2018-04-04
38130    2018-04-25
30402    2018-11-07
38885    2018-12-31
26559    2020-12-16
Name: release_date, Length: 45363, dtype: object

In [117]:
# De haber fechas, deberán tener el formato AAAA-mm-dd, 
# además deberán crear la columna release_year donde extraerán el año de la fecha de estreno.
# Se crea tambien las columnas release_month y release_day para ser utilizadas en los metodos de la api.
df['release_date'] = df['release_date'].astype(str)

df['release_date']= pd.to_datetime(df['release_date']) 

df['release_year'] = df['release_date'].dt.year 
df['release_month'] = df['release_date'].dt.month 
df['release_day'] = df['release_date'].dt.day 
# verificar algunos registros
df.head(10)

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,release_year,release_month,release_day
0,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000.0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,...,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,7.7,5415.0,1995,10,30
1,NaN,65000000.0,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,en,When siblings Judy and Peter discover an encha...,17.015539,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,...,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,1995,12,15
2,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0.0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602,en,A family wedding reignites the ancient feud be...,11.7129,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,...,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0,1995,12,22
3,NaN,16000000.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,...,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,6.1,34.0,1995,12,22
4,"{'id': 96871, 'name': 'Father of the Bride Col...",0.0,"[{'id': 35, 'name': 'Comedy'}]",11862,en,Just when George Banks has recovered from his ...,8.387519,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,...,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,173.0,1995,2,10
5,NaN,60000000.0,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",949,en,"Obsessive master thief, Neil McCauley leads a ...",17.924927,"[{'name': 'Regency Enterprises', 'id': 508}, {...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,...,170.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,A Los Angeles Crime Saga,Heat,7.7,1886.0,1995,12,15
6,NaN,58000000.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...",11860,en,An ugly duckling having undergone a remarkable...,6.677277,"[{'name': 'Paramount Pictures', 'id': 4}, {'na...","[{'iso_3166_1': 'DE', 'name': 'Germany'}, {'is...",1995-12-15,...,127.0,"[{'iso_639_1': 'fr', 'name': 'Français'}, {'is...",Released,You are cordially invited to the most surprisi...,Sabrina,6.2,141.0,1995,12,15
7,NaN,0.0,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",45325,en,"A mischievous young boy, Tom Sawyer, witnesses...",2.561161,"[{'name': 'Walt Disney Pictures', 'id': 2}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,...,97.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,The Original Bad Boys.,Tom and Huck,5.4,45.0,1995,12,22
8,NaN,35000000.0,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",9091,en,International action superstar Jean Claude Van...,5.23158,"[{'name': 'Universal Pictures', 'id': 33}, {'n...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,...,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Terror goes into overtime.,Sudden Death,5.5,174.0,1995,12,22
9,"{'id': 645, 'name': 'James Bond Collection', '...",58000000.0,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",710,en,James Bond must unmask the mysterious head of

In [120]:
df.to_csv('../movies_pimlops/dataset/dataset_limpio.csv', index=False)